## Machine Learning
Try deep learning netwrok for binary classification of the metabolon data

In [53]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
import tensorflow as tf




In [54]:
X_fwn = pd.read_csv('muscle_adjusted_values.csv') # reading data
y = pd.DataFrame()
# y['Ex'] = X_fwn.pop('Ex')
y['Tx'] = X_fwn.pop('Tx')
X_fwn = X_fwn.drop(['Group','MouseID','Ex'], axis=1)
X_fwn.head()

,oxoadipate,kynurenate,Nformylmethionine,AMP,adenosinediphosphoriboseADPribose,hydroxyphenyllactateHPLA,betaalanine,carnosine,phosphocholine,creatinine,...,hydroxydecanoylcarnitine,succinoyltaurine,hexanoyltaurine,Nacetylaminoadipate,ditertbutylphenol,NNdimethylpropro,oxindolylalanine,oleylGPCO,avg_dVO2,avg_VO2
0,-1.467669,-0.502356,0.604020,1.155266,0.554374,0.509589,-1.682942,-0.081640,-2.183583,0.550942,...,3.497966,-1.146348,-1.230095,-1.129802,-2.074813,-0.420198,-2.253050,0.949912,1.622066,-0.585547
1,0.812456,-0.729896,1.735956,1.902503,-1.354648,0.655030,-0.986382,-0.271350,-0.577402,-0.863437,...,1.775979,-0.561337,-1.104361,-0.734704,-0.781902,0.299352,-1.856883,-0.588613,1.450905,0.939831
2,0.341596,0.520124,-0.080949,1.064611,-1.072894,0.630093,2.124104,-0.199274,0.007674,0.149239,...,1.428994,0.562333,0.303499,0.525386,-0.173917,-0.767051,-1.005226,0.647059,0.556003,-0.499269
3,0.696459,0.215898,-0.033017,-0.300402,-1.426641,-0.367192,0.047762,0.162822,0.566678,1.063032,...,1.555703,-0.737090,-0.302112,0.749926,-0.775116,0.393152,-0.293635,-0.035675,0.496940,0.196349
4,0.189109,-0.392664,-0.408489,1.392754,1.670193,-1.795812,-0.346030,0.315041,0.347505,-0.124541,...,-0.077946,-1.270169,-1.126717,-0.509758,0.061403,-0.051534,-1.379212,-0.882308,0.761212,0.854730


In [55]:
enc = OneHotEncoder() # creating instance of one-hot encoding function from sklearn
y_fit = enc.fit_transform(y).toarray() # converting Ex and Tx columns to binary
# y['Ex'] = y_fit[:,0] # assigning in correct order so that 1 = runner..
# y['Tx'] = y_fit[:,2] # and 1 = 2HB
y['Tx'] = y_fit[:,0]

In [56]:
y

,Tx
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,1.0
8,1.0
9,1.0


In [57]:
input_shape = [X_fwn.shape[1]]
model = keras.Sequential([
    layers.Dense(64,activation = 'relu',input_shape = input_shape),
    layers.Dropout(0.3),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1,activation='sigmoid')
])

In [58]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['binary_accuracy'])

In [59]:
kf = KFold(n_splits=3,shuffle=True)


Fold 0:
  Train: index=[ 0  1  3  4  5  6  7  9 10 13 15 19 20 21 22 23]
  Test:  index=[ 2  8 11 12 14 16 17 18]
Fold 1:
  Train: index=[ 1  2  4  5  7  8 11 12 13 14 15 16 17 18 20 22]
  Test:  index=[ 0  3  6  9 10 19 21 23]
Fold 2:
  Train: index=[ 0  2  3  6  8  9 10 11 12 14 16 17 18 19 21 23]
  Test:  index=[ 1  4  5  7 13 15 20 22]


In [61]:
pred = model.predict(X_fwn)

1/1 [==============================] - 0s 55ms/step


In [62]:
pred.round()

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [63]:
from sklearn.model_selection import GridSearchCV 
#simple performance reporting function - from Ken Jee
def grid_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [64]:
"""
xgb = XGBClassifier()
param_grid = {
    'n_estimators': [30, 50, 70],
    'colsample_bytree': [0.05, 0.1, 0.15],
    'max_depth': [2, 5, 8],
    'reg_alpha': [1, 1.5],
    'reg_lambda': [0.5, 1],
    'subsample': [0.9, 1],
    'learning_rate':[.05, 0.1, 0.15],
    'gamma':[1,5,10],
    'min_child_weight':[0],
    'sampling_method': ['uniform']
}
grid_xgb = GridSearchCV(xgb, param_grid = param_grid, cv = 3, verbose = True, n_jobs = -1)
# verbosity just controls how much feedback the program outputs for each candidate tested.
# n_jobs = -1 means that we test all possible combinations based on hte range of inputs we gave
best_xgb = grid_xgb.fit(X_fwn,y)
grid_performance(best_xgb,'XGBoost')
"""
print("XGBoost, Best Score: 0.9583333333333334, \
    Best Parameters: {'colsample_bytree': 0.1, 'gamma': 1, 'learning_rate': 0.05, 'max_depth': 2, \
    'min_child_weight': 0, 'n_estimators': 50, 'reg_alpha': 1.5, 'reg_lambda': 1, \
    'sampling_method': 'uniform', 'subsample': 0.9}'")

XGBoost, Best Score: 0.9583333333333334,     Best Parameters: {'colsample_bytree': 0.1, 'gamma': 1, 'learning_rate': 0.05, 'max_depth': 2,     'min_child_weight': 0, 'n_estimators': 50, 'reg_alpha': 1.5, 'reg_lambda': 1,     'sampling_method': 'uniform', 'subsample': 0.9}'


In [66]:
earlystopping = keras.callbacks.EarlyStopping(
    patience=20,
    min_delta=0.0001,
    restore_best_weights=True
)

features = X_fwn.columns.isin(X_fwn.columns) # initialize feature selection with all as true
#initial model to find best_loss score for initial target
input_shape = [features.sum()]
model = keras.Sequential([
    layers.Dense(64,activation = 'relu',input_shape = input_shape),
    layers.Dropout(0.3),
    layers.Dense(64, activation = 'relu'),
    layers.Dropout(0.3),
    layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics = ['binary_accuracy'])
# test the model with 3 folds
kf = KFold(n_splits=3,shuffle=True)
for j, (train_index, test_index) in enumerate(kf.split(X_fwn)):
    model.fit(
    X_fwn.iloc[train_index,features],y.iloc[train_index,:],
    epochs = 200,
    verbose = 0,
    validation_data=(X_fwn.iloc[test_index,features],y.iloc[test_index,:]),
    callbacks=[earlystopping]
    )
    best_loss = min(best_loss, model.get_metrics_result()['loss'].numpy())
print(best_loss)

for i, met in enumerate(X_fwn.columns):
    met_loss = 10000
    for j, (train_index, test_index) in enumerate(kf.split(X_fwn)):
        input_shape = [features.sum()]
        model = keras.Sequential([
        layers.Dense(64,activation = 'relu',input_shape = input_shape),
        layers.Dropout(0.3),
        layers.Dense(64, activation = 'relu'),
        layers.Dropout(0.3),
        layers.Dense(1,activation='sigmoid')
        ])
        model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = ['binary_accuracy'])
        model.fit(
        X_fwn.iloc[train_index,features],y.iloc[train_index,:],
        epochs = 200,
        verbose = 0,
        validation_data=(X_fwn.iloc[test_index,features],y.iloc[test_index,:]),
        callbacks=[earlystopping]
        )
        met_loss = min(met_loss, model.get_metrics_result()['loss'].numpy())
    if met_loss <= best_loss:
        features[i] = False
    print(i,met,met_loss,features.sum())


0.0013269183
0 oxoadipate 0.057735324 529
1 kynurenate 0.11092714 529
2 Nformylmethionine 0.06048448 529
3 AMP 0.6638415 529
4 adenosinediphosphoriboseADPribose 0.34743387 529
5 hydroxyphenyllactateHPLA 0.2610006 529
6 betaalanine 0.3489922 529
7 carnosine 0.47601575 529
8 phosphocholine 0.21818027 529
9 creatinine 0.34511146 529
10 CMP 0.20312944 529
11 dihydroxyacetonephosphateDHAP 0.053884167 529
12 FAD 0.41341507 529
13 gammaglutamylglutamate 0.21491526 529
14 gluconate 0.71917486 529
15 glutarateCDC 0.07947758 529
16 glycine 0.1754455 529
17 GMP 0.21481636 529
18 hypotaurine 0.13786124 529
19 inosinemonophosphateIMP 0.1049355 529
20 citrulline 0.3101343 529
21 methionine 0.8265809 529
22 orotate 0.43921596 529
23 glutathioneoxidizedGSSG 0.44185477 529
24 phenylalanine 0.5449822 529
25 phosphate 0.1123845 529
26 proline 0.52237105 529
27 lactate 0.07966341 529
28 serine 0.23066866 529
29 serotonin 0.14223865 529
30 taurine 0.23443416 529
31 deoxyuridine 0.23775719 529
32 glutamine 

KeyboardInterrupt: 

230619 - made it as far as 215 Malonate, still all 529 features are included